In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1)


* nn.Module
    * nn.Module을 상속하여, 사용자 정의 model class를 만들 수 있다.
    * module에 함수 계산만 정의해두면, 계산값과 gradient값을 반환한다.
    * nn.Linear, nn.Sigmoid 등의 함수는 미리 정의된 nn.Module이라 생각하면 된다.
        * 이런 미리 정의된 함수는 gradient를 반환하기에 연속으로 사용 불가능 하지만(logistic_regression함수 참고), nn.Module을 상속한 class에선 연속으로 사용 가능하다.

* torch 함수
    * torch.tensor.scatter_
    * torch.squeeze(unsqueeze) 
        * squeeze: size가 1인 dimension을 제거, ex) A1BC1D -> ABCD
        * unsqueeze(x,k): x에서 k번째 dimension에 size가 1인 dimension 추가
            * ex) unsqueeze(x,1): ABC -> A1BC, unsqueeze(x,0): ABC -> 1ABC        
    * F.cross_entropy(linear regression, y_arr)
        * softmax classification은 linear regression-> softmax -> cross_entropy의 연산을 해야하며, label도 one_hot으로 바꿔주어야 한다.
        * cross_entropy함수는 복잡한 softmax연산과 one_hot변환조차 생략할 수 있게 해준다.

* 함수 설명
    * logistic regression
        * prediction = nn.Sigmoid(model(x_train)) 은 Linear model에서 2개의 값을 반환해서(결과, gradient) Sigmoid와 이어서 사용할 수 없다.
    * logistic regression2 
        * nn.Module을 상속한 class에서는 이유는 모르겠는데?(아마 super.__init__덕분에) Linear model과 Sigmoid를 이어서 사용할 수 있다. 
    * softmax classification
        * cross_entropy를 이용하여 softmax연산과 one_hot변경을 생략하여 함수가 엄청 간단해졌다.

In [ ]:
def logistic_regression():
    x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
    y_data = [[0], [0], [0], [1], [1], [1]]

    x_train = torch.FloatTensor(x_data)
    y_train = torch.FloatTensor(y_data)

    # W = torch.zeros((2,1),requires_grad=True)
    # b = torch.zeros(1,requires_grad=True)
    # optimizer = optim.SGD([W,b],lr=1)
    model = nn.Linear(2, 1)
    optimizer = optim.SGD(model.parameters(), lr=1)

    epochs = 1000
    for epoch in range(1 + epochs):
        # prediction = 1/(1+torch.exp(-(x_train.matmul(W)+b)))
        # cost = -(y_train*torch.log(prediction)+(1-y_train)*torch.log(1-prediction)).mean()
        print(model(x_train))
        prediction = nn.Sigmoid(model(x_train))
        cost = F.binary_cross_entropy(prediction, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if epoch % 100 == 0:
            prediction = prediction >= torch.FloatTensor([0.5])
            print(
                "Epoch: {:4d}/{}, Prediction: {} Cost: {:.6f}".format(
                    epoch, epochs, prediction.squeeze().detach(), cost.item()
                )
            )

    # print(W,"\n",b)
    print(list(model.parameters())[0], "\n", list(model.parameters)[1])


logistic_regression()


In [ ]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()  # nn.Module의 __init__ 을 실행
        # self.linear = nn.Linear(2,1)
        # self.sigmoid = nn.Sigmoid()
        self.model = nn.Sequential(nn.Linear(2, 1), nn.Sigmoid())

    def forward(self, x):
        # return self.sigmoid(self.linear(x))
        return self.model(x)


def logistic_regression2():
    x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
    y_data = [[0], [0], [0], [1], [1], [1]]

    x_train = torch.FloatTensor(x_data)
    y_train = torch.FloatTensor(y_data)

    model = BinaryClassifier()
    optimizer = optim.SGD(model.parameters(), lr=1)

    epochs = 1000
    for epoch in range(1 + epochs):
        prediction = model(x_train)
        cost = F.binary_cross_entropy(prediction, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if epoch % 100 == 0:
            prediction = prediction >= torch.FloatTensor([0.5])
            correct_prediction = prediction.float() == y_train
            accuracy = correct_prediction.sum().item() / len(correct_prediction) * 100
            print(
                "Epoch: {:4d}/{}, Accuracy: {}% Cost: {:.6f}".format(
                    epoch, epochs, accuracy, cost.item()
                )
            )

    # print(W,"\n",b)
    print(list(model.parameters()))


logistic_regression2()


In [66]:
# class SoftmaxClassifier(nn.Module):
#     # nn.Linear만 있을거면 없어도 된다.
#     def __init__(self):
#         self.super().__init__()
#         self.linear = nn.Linear(4, 3)

#     def forward(self, x):
#         return self.linear(x)


def softmax_classification():
    x_data = [
        [1, 2, 1, 1],
        [2, 1, 3, 2],
        [3, 1, 3, 4],
        [4, 1, 5, 5],
        [1, 7, 5, 5],
        [1, 2, 5, 6],
        [1, 6, 6, 6],
        [1, 7, 7, 7],
    ]
    y_data = [2, 2, 2, 1, 1, 1, 0, 0]
    x_train = torch.FloatTensor(x_data)
    y_train = torch.LongTensor(y_data)

    # F.cross_entropy를 사용하면 필요없다.
    # y_onehot = torch.zeros(8, 3)
    # y_onehot.scatter_(1, y_train.unsqueeze(1), 1)

    # W = torch.zeros((4, 3), requires_grad=True)
    # b = torch.zeros(1, requires_grad=True)
    # optimizer = optim.SGD([W, b], lr=0.1)
    model = nn.Linear(4, 3)
    optimizer = optim.SGD(model.parameters(), lr=0.1)

    epochs = 1000
    for epoch in range(epochs + 1):
        # softmax와 cross entropy를 정석대로 구현
        # prediction = F.softmax(x_train.matmul(W)+b, dim=1) # dim=1: dim=1 내부에서 softmax, not 전체
        # cost = (y_onehot* -torch.log(prediction)).sum(dim=1).mean() # dim=1: dim=1 내부에서 sum, not 전체
        # cross_entropy 함수를 이용해 one_hot label도 안만들고, softmax도 직접구현 안함
        # z = x_train.matmul(W) + b
        # cost = F.cross_entropy(z, y_train)
        # class를 이용해 더 간소화
        prediction = model(x_train)
        cost = F.cross_entropy(prediction, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print("Epoch: {:4d}/{}, Cost: {:.6f}".format(epoch, epochs, cost.item()))


softmax_classification()


Epoch:    0/1000, Cost: 2.823855
Epoch:  100/1000, Cost: 0.712041
Epoch:  200/1000, Cost: 0.623341
Epoch:  300/1000, Cost: 0.564995
Epoch:  400/1000, Cost: 0.514439
Epoch:  500/1000, Cost: 0.466889
Epoch:  600/1000, Cost: 0.420623
Epoch:  700/1000, Cost: 0.374839
Epoch:  800/1000, Cost: 0.329233
Epoch:  900/1000, Cost: 0.284424
Epoch: 1000/1000, Cost: 0.246721
